<a href="https://colab.research.google.com/github/advapplab/esgBERT_hf/blob/main/MLM_evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install awscli

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.9 MB 5.8 MB/s 
     |████████████████████████████████| 79 kB 6.7 MB/s 
     |████████████████████████████████| 9.9 MB 32.1 MB/s 
     |████████████████████████████████| 636 kB 44.4 MB/s 
     |████████████████████████████████| 548 kB 55.9 MB/s 
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
  Attempting uninstall: docutils
    Found existing installation: docutils 0.17.1
    Uninstalling docutils-0.17.1:
      Successfully uninstalled docutils-0.17.1


In [15]:
import os

!export AWS_SHARED_CREDENTIALS_FILE = /content/awscli.ini

path = "/content/awscli.ini"
os.environ['AWS_SHARED_CREDENTIALS_FILE'] = path
print(os.environ['AWS_SHARED_CREDENTIALS_FILE'])

/bin/bash: line 0: export: `=': not a valid identifier
/bin/bash: line 0: export: `/content/awscli.ini': not a valid identifier
/content/awscli.ini


In [17]:
!aws s3 cp s3://esg-bert-pretraining/pretraining_testingdata.csv

Note: AWS CLI version 2, the latest major version of the AWS CLI, is now stable and recommended for general use. For more information, see the AWS CLI version 2 installation instructions at: https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2.html

usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help
aws: error: the following arguments are required: paths


In [1]:
!pip install transformers datasets pandas numpy evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.2 MB/s 
     |████████████████████████████████| 451 kB 52.7 MB/s 
     |████████████████████████████████| 72 kB 1.2 MB/s 
     |████████████████████████████████| 182 kB 54.2 MB/s 
     |████████████████████████████████| 7.6 MB 24.8 MB/s 
     |████████████████████████████████| 212 kB 64.7 MB/s 
     |████████████████████████████████| 115 kB 45.6 MB/s 
     |████████████████████████████████| 127 kB 53.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [22]:
import datasets
import pandas as pd
input_df = pd.read_csv('pretraining_testingdata.csv', encoding = 'ISO-8859-1')
input_df = input_df[['content']]

raw_datasets = datasets.DatasetDict(
    {
        "evaluate": datasets.Dataset.from_pandas(input_df)
    }
)

In [23]:
from transformers import BertTokenizer
context_length = 128
model_checkpoint = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_checkpoint)

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_positio

In [24]:
def tokenize_function(texts):
    return tokenizer(texts["content"], padding="max_length", truncation=True) # max_length=context_length

tokenized_datasets = raw_datasets.map(tokenize_function)

  0%|          | 0/300 [00:00<?, ?ex/s]

In [25]:
tokenized_datasets

DatasetDict({
    evaluate: Dataset({
        features: ['content', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 300
    })
})

In [26]:
eval_dataset = tokenized_datasets['evaluate'].shuffle(seed=42).select(range(30))
eval_dataset

Dataset({
    features: ['content', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 30
})

In [7]:
from transformers import BertForMaskedLM
model = BertForMaskedLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

In [9]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

In [10]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, targets = eval_pred.predictions, eval_pred.label_ids
    maxProbInds = np.argmax(predictions, axis=-1)
    
    mask_pos_preds =  maxProbInds[tuple([targets != -100])]
    mask_pos_targets = targets[tuple([targets != -100])]
    
    # python (no numpy)
    # mask_pos_preds, mask_pos_targets = [], []
    # seq_count, seq_len = targets.shape
    # for s in range(seq_count):
    #     for t in range(seq_len):
    #         if targets[s][t] != -100:
    #             mask_pos_preds.append(maxProbInds[s][t])
    #             mask_pos_targets.append(targets[s][t])
    
    return metric.compute(predictions=mask_pos_preds, references=mask_pos_targets)

In [27]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: content. If content are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 30
  Batch size = 8


{'eval_loss': 2.3396520614624023,
 'eval_accuracy': 0.587958607714017,
 'eval_runtime': 75.528,
 'eval_samples_per_second': 0.397,
 'eval_steps_per_second': 0.053}